# Melody Generation

+ **AI in Culture and Arts - Tech Crash Course**
+ **Date:** 06.06.2024
+ **Author:** B. Zönnchen

<a href="https://colab.research.google.com/github/aica-wavelab/aica-assignments/blob/main/A4_melody_generation/2_2_style_imitation.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In the following we will create music sheets and sound. For those tasks ``Python`` requires external programs that you should install if you are working locally:

1. [Musescore](https://musescore.org/de) (for generating sheets)
2. [FluidSynth](https://www.fluidsynth.org/) (for generating sound)

If you are working on google ``Colab`` you can evaluate the following three cells to install these applications:

In [ ]:
#@title install dependencies to play sound
%%capture
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done!')

In [ ]:
#@title install dependencies to show score in music notation
%%capture
print('installing musescore3...')
!apt-get install musescore3 > /dev/null
print('done!')

In [ ]:
#@title clone git repository
%%capture
%rm -rf aica-assignments
!git clone https://github.com/aica-wavelab/aica-assignments.git
%cd aica-assignments/A4_melody_generation

Furtheremore, for this notebook, we need the following ``Python`` packages and moduls. Execute the cell to install them:

In [ ]:
%pip install music21
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install pyfluidsynth
%pip install otter-grader

In [ ]:
import random
import music21 as m21
from music21.note import Note, Rest
from music21.chord import Chord
from music21.stream import Part, Score
import numpy as np

from encoder import NoteToIntEncoder
from encoder import TERM_SYMBOL

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("2_2_style_imitation.ipynb")

In [ ]:
# from notebook 2_genuine_composing
def generate_random_score(length=10, lower=m21.note.Note('C3'), upper=m21.note.Note('C6'), q=0.1):
  score = m21.stream.Score()
  part = m21.stream.Part()

  for _ in range(length):
    
    # with probability q there will be a rest
    if random.random()<=q:
      part.append(m21.note.Rest())
    else:
      midi = random.randint(lower.pitch.midi, upper.pitch.midi)
      note = m21.note.Note()
      note.pitch.midi = midi
      part.append(note)
  score.insert(0, part)
  return score

## 2.2 Style Imitation

Let us suppose we have no music theory knowledge and we still want to generate a melody that sounds rather pleasing.
And let us suppose we have some melodies from well-known composers at our fingertips.
In this case we can learn from the data! That is, we can apply *machine learning* techniques.

### 2.2.1 Learning Intervals

We can, for example, analyse which intervals the composer uses and we can compute how often a certain interval was used.
The advantage of this approach is that we do not rely on certain note combinations.

We exploit a property that is very special to musical notations: **translation does keep the essence of the piece alive**.
This is very different compared to text, since if we translate a text by one letter, the meaning is completely different!
**Note:** Translation works only well for an *equal temperament instrument*.

Let us implement a function that returns the frequencies of intervals in a ``dict``.
We might want to ignore *inversions* (shifting a pitch one or multiple octaves).
To iterate over each element in a ``Stream`` object (e.g. ``Part`` or ``Score``) we can use the ``.recurse()`` method of that object.
You can find further information [here](https://web.mit.edu/music21/doc/usersGuide/usersGuide_06_stream2.html).

Let us first test this by printing all the notes of a ``Score``, luckily we can now use our function above to compute some random ``Score``!

In [ ]:
score = generate_random_score()
for element in score.recurse():
  print(element)

Ok, as we can see the iteration also goes over ``Part``. We can assume that our score is monophonic, thus we are only interested in ``Note``s.
Using the ``Python`` function ``isinstance`` we can check for the ``type`` (and subtype) of the object:

In [ ]:
score = generate_random_score()
for element in score.recurse():
  if isinstance(element, Note):
    print(element)

Nice! We only get the ``Note``s. Let us implement the function ``count_intervals(score, ignore_inversions, intervals=None)``. ``score`` is the ``Score`` we want to extract intervals from,``ignore_inversions`` indicates if we do not want to differentiate between e.g. ``C4`` and ``C5`` and the optional ``intervals`` argument is the result of calling this function on some other ``Score`` which makes it possible to consider multiple ``Score``s.

In [ ]:
# the function can optionally receive a non-empty dictionary. This is useful if we want to apply it for multiple scores.
def count_intervals(score, ignore_inversions=False, intervals = None):

  if intervals == None:
    intervals = dict()

  # we start with nothing!
  current = None
  
  # we only use the first part assuming it is the melody in the piece.
  for element in score.parts[0].recurse():
    if isinstance(element, m21.note.Note):
      # for the first element we cannot compute an interval
      if current != None:

        # compute the interval
        interval = element.pitch.midi - current.pitch.midi

        # if we want to ignore inversions, we use the modulo operator
        if ignore_inversions:
          interval = interval % 12
        
        # if the interval has already an entry in the dictionary, we just add 1 otherwise we have to create a new entry
        if interval in intervals:
          intervals[interval] += 1
        else:
          intervals[interval] = 1
      current = element
  return intervals

In [ ]:
intervals = count_intervals(score, True)
intervals

In [ ]:
intervals = count_intervals(score, False)
intervals

Next we need a nice data structure to pick intervals from.
More frequent intervals should be picked more often.
Therefore, we can compute a discrete probability distribution over the intervals.
First we convert the frequencies in probabilities.
We can do this by 

1. computing the number overall number of intervals
2. divide each value in the ``dict`` by this number
3. extract everything into a list of pairs ``(interval, probability)``
4. sort it by the probability in descending order

In [ ]:
def to_probability_dist(intervals):
  # 1. computing the number of intervals
  n_intervals = 0
  for n in intervals.values():
    n_intervals += n
  n_intervals

  # 2. divide each value in the ``dict`` by this number
  for key in intervals.keys():
    intervals[key] /= n_intervals
  intervals

  # 3. extract everything into a list of pairs
  intervals_list = []
  for item in intervals.items():
    intervals_list.append(item)

  # 4. sort it by the probability
  intervals_list.sort(key=lambda item: item[1], reverse=True)
  return intervals_list

In [ ]:
prob_list = to_probability_dist(intervals)
prob_list

Now we can generate a random number $r$ in $[0;1)$ and go through the list until the sum of probabilities is larger or equal to $r$.
This is the interval we have to use! 

In [ ]:
def choose_interval(prob_list):
  r = random.random()
  p = 0
  i = 0
  while r >= p:
    p += prob_list[i][1]
    i += 1
  return prob_list[i-1][0]

choose_interval(prob_list)

Now everything is in place and we can 

1. compute our probability list by analysing one or multiple scores
2. generate a new score using ``choose_interval``.

There is only one last problem: Nothing prevents us to move high up or down the octaves! A simple solution is to define a certain interval we have to be in and to use inversion if this is not the case!

Let us use a single function to do everything!

In [ ]:
# compute our probability list by analysing multiple scores
def interval_dist(scores, ignore_inversions=False):
  interval_freqs = dict()
  for score in scores:
    interval_freqs = count_intervals(score, ignore_inversions=ignore_inversions, intervals=interval_freqs)
  prob_list = to_probability_dist(interval_freqs)
  return prob_list

def generate_score_by_interval_dist(interval_dist, 
                         tonic_note=Note('C4'), 
                         length=10,
                         lowest_note=Note('C6'), 
                         highest_note=Note('C2')):
  score = m21.stream.Score()
  part = m21.stream.Part()
  
  # 2. add the tonic / root note
  note = tonic_note
  part.append(note)

  for _ in range(length-1):
    # 3. pick the next interval and generate the next note
    interval = choose_interval(interval_dist)
    note = m21.note.Note()
    note.pitch.midi = note.pitch.midi + interval

    # 4. sanity inversion to push the next note in our desired range
    distance = note.pitch.midi - highest_note.pitch.midi
    if distance > 0:
      # // is a division of whole numbers, e.g. 7.6 // 2.0 == 3.0
      note.pitch.midi -= (distance // 12 + 1) * 12

    distance = lowest_note.pitch.midi - note.pitch.midi
    if distance > 0:
      # // is a division of whole numbers, e.g. 7.6 // 2.0 == 3.0
      note.pitch.midi += (distance // 12 + 1) * 12
    
    part.append(note)
  score.insert(0, part)
  return score

In [ ]:
# 1. generate a bad "training dataset"
score = generate_random_score(10)
# 2. train our model
model = interval_dist([score], True)
gen_score = generate_score_by_interval_dist(model, Note('C4'), 10, Note('C3'), Note('C6'))
gen_score.show('midi')

We can use a subset of the ``m21.corpus`` as our training dataset. However, it does not contain simple melodies. Therefore, we can try to identify monophonic pieces (i.e. without ``Chord``s) and only consider the first ``Part``. Note that this dataset is not that great!
The following function ``is_monophonic(score)`` returns true if the ``Score`` does not contain any ``Chord``s.

In [ ]:
def is_monophonic(score):
    """Check if a score is monophonic."""
    for part in score.parts:
        previous_offset = None
        for note in part.flatten().notes:
            if isinstance(note, Chord):
                return False
            if previous_offset is not None and note.offset == previous_offset:
                return False
            previous_offset = note.offset
    return True

Ok, now we can try to imitate other composers. Let us filter the corpus to get all monophonic pieces by Bach:

In [ ]:
training_dataset = []
monophonic_bach_works = m21.corpus.search(composer='bach')

# this can take some time
for work in monophonic_bach_works:
  score = work.parse()
  if is_monophonic(score):
    training_dataset.append(score)

print(f'There are {len(training_dataset)} monophonic works of Bach')

In [ ]:
training_dataset[0].show('midi')

Let us *train* our model:

In [ ]:
model = interval_dist(training_dataset, False)
model

Let us generate a new score! Note that we only consider intervals and no durations!

In [ ]:
gen_score = generate_score_by_interval_dist(model,Note('C4'), 40, Note('C3'), Note('C6'))
gen_score.show('midi')

Let's compare it with our random score generation:

In [ ]:
ran_score = generate_random_score(40)
ran_score.show('midi')

### 3.2 Learning Note-Transitions with Markov Chains

Sice we are learning only the probability distribution of intervals, we only use very little information of a corpus. As we saw, this does not really lead to good results.

Next we want to learn the probability of appearence of the next note (pitch & duration) based on its predecessor.
We can do this by learning a so called (time-homogeneous) *[discrete Markov chain](https://en.wikipedia.org/wiki/Markov_chain)*.

A melody can be represented by a set of notes $\{n_1, n_2, \ldots, n_m\}$ (including rests) defined by their *pitch* and *duration*. For each note $n_i$ we can compute the frequency of transitions to the next note. 
For example, take the following sequence: ``C4 C4 F4 D4 F4 D4 C4 C4`` and assume all durations are the same, then we would have 

+ (``TERM_SYMBOL`` -> ``C4``) 1 times,
+ (``C4`` -> ``C4``) 2 times, 
+ (``C4`` -> ``F4``) 1 times,
+ (``F4`` -> ``D4``) 2 times,
+ (``D4`` -> ``F4``) 1 times,
+ (``D4`` -> ``C4``) 1 times,
+ (``C4`` -> ``TERM_SYMBOL``) 1 times

Thus we know that if a ``C4`` is played a ``C4`` will follow with probability ``0.5``, or the piece will end with probability ``0.25`` or a ``F4`` will follow with probability ``0.25``.

To generate a piece of music we start with ``TERM_SYMBOL``. It is a special symbol that marks the start and end of a melody. Then we roll the dice and pick the next note according to the probabilities of available transitions.
Then we roll the dice again and pick the next note until we reach the ``TERM_SYMBOL`` again.

So far we worked directly with the objects of ``music21``. It is however often advisable to work with numbers instead. Our problem is one such situation. Instead of working with notes $\{n_1, n_2, \ldots, n_m\}$ we could work instead with numbers $0,1, \ldots, m-1$, where each number represents the respective note. We implemented the ``NoteToIntEncoder`` that can be used to do this encoding. It expects a list of ``music21`` ``Stream`` objects to compute the set of notes. The ``encoder`` maps ``Note``s and ``Rest``s to consecutive natural numbers starting from ``0``

In [ ]:
part = Part()
part.append([
  Note('C4'), Note('A4'), Rest(0.5), 
  Note('C5'), Rest(1.0), Note('C4'), 
  Rest(1.0), Note('A4')
  ])

encoder = NoteToIntEncoder([part, part])

for element in part:
  print(f'{element.fullName} -> {encoder.encode(element)} -> {encoder.decode(encoder.encode(element))}')

Calling the ``len(encoder)`` returns the number of unique elements, i.e. $m-1$:

In [ ]:
m = len(encoder)
print(m)

One simple representation for the probability transitions is the so-called *Markov* matrix.
Each row of the matrix represents the starting point of a transition and each column endpoint of a transition.
Therefore, the value in row $i$ and column $j$ tells us the probability from the state / note $i$ to the state / note $j$.
The following is an example of a Markov matrix with $m=3$ states / notes.

$$
M = \begin{pmatrix} 0.1 & 0.3 & 0.6 \\ 0.3 & 0.3 & 0.4 \\ 0.0 & 0.1 & 0.9 \end{pmatrix}
$$

Note that the sum over all elements of a row has to be equal to ``1.0``.

The following code computes the Markov matrix. It requires the respective ``Stream``s and the respective ``NoteToIntEncoder``. 

In [ ]:
def markov_matrix(streams, encoder):
  m = len(encoder)

  # initiate the Markov matrix with zeros
  M = np.zeros((m,m))

  # count the transitions; a row represents the starting point and the column the end point
  for part in streams:
    predecessor = TERM_SYMBOL
    idx2 = 0
    for element in part.recurse():
      if isinstance(element, (m21.note.Rest, m21.note.Note)):
        idx1 = encoder.encode(predecessor)
        idx2 = encoder.encode(element)
        M[idx1][idx2] += 1.0
        predecessor = element
      # this is for the ending since we arrive at a TERM_SYMBOL
    M[idx2][encoder.encode(TERM_SYMBOL)] += 1.0

  # divide each element of a row by the sum of that row
  return M / M.sum(axis=1, keepdims=True)

To generate a ``Score``, we start by setting our first element / event / state to ``TERM_SYMBOL``.
Then we pick the row from the Markov matrix ``M`` of that element which gives us a probability distribution.
According to this distribution we pick the next element / event / state which we add to our ``Score``.
We repeat this as long as the score length is smaller than ``max_len`` or we run into ``TERM_SYMBOL`` again.

In [ ]:
def generate_markov_score(M, encoder, max_len=100):
  # start with the terminal symbol, i.e. with the start of a piece
  score = Score()
  part = Part()
  
  element = encoder.encode(TERM_SYMBOL)
  for _ in range(max_len):
    m = len(encoder)

    # pick the next element according to the distribution M[element]
    element = np.random.choice(m, size=1, p=M[element])[0]

    # decode the number element to a symbol
    symbol = encoder.decode(element)

    # check if we should stop
    if symbol == TERM_SYMBOL:
      break
    else: 
      part.append(symbol)
      
  score.insert(0, part)
  return score

Let us try our approach using only parts of Bach's *Minuet in G*.

In [ ]:
bach_minuet = m21.converter.parse('data/Minuet_in_G.mid')
bach_minuet.show('midi')

We extract the first part and we ignore all ``Chord``s.

In [ ]:
bach_minuet_melody = bach_minuet.parts[0]
bach_minuet_melody.show('midi')

Again, the ``NoteToIntEncoder`` encodes the notes of ``Stream``s to natural numbers in $[0;m-1]$.
First we generate our Markov matrix ``M``.

In [ ]:
encoder = NoteToIntEncoder([bach_minuet_melody])
M = markov_matrix([bach_minuet_melody], encoder)

According to ``M``, we generate a new melody:

In [ ]:
minuet_imitation = generate_markov_score(M, encoder=encoder, max_len=10_000)

In [ ]:
minuet_imitation.show('midi')

<!-- BEGIN QUESTION -->

We can clearly hear the similarities of Bach's *Minuet in G* and our piece. Of course, this is musically not very interesting. Often, Markov chains are used either for ambient music or for transitions that are not based on notes but on more large scale concepts such as themes or musical ideas.

<div class="alert alert-info">

**Instruction 2.2.1** Our Markov chain is of order 1, meaning that the next note only depends on the previous one. One can say that this process is memoryless.

1. How could we change our implementation to make the next ``Note`` dependent on the last two previous ``Note``s. Describe it conceptually, you do not have to describe the required code!
2. Let's suppose the next note depends on the last $n$ notes. Would this result in a better or worse imitation?

</div>

_Type your answer here, replacing this text._

<!-- END QUESTION -->



---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()